In [0]:
USE CATALOG dbdemos_vishesh;
USE SCHEMA contest_mock;

SHOW CREATE TABLE contests_view;

CREATE OR REPLACE VIEW contests_metric_view
(
  `Contest ID` COMMENT 'A unique identifier for each contest, allowing for easy reference and tracking of specific contests.',
  `Contest Sport` COMMENT 'Indicates the sport associated with the contest, which helps in categorizing and filtering contests based on the sport type.',
  `Contest Type` COMMENT 'Describes the type of contest, providing insight into the format or rules governing the contest.',
  `Contest Create Date` COMMENT 'Records the date when the contest was created, useful for understanding the timeline of contest availability.',
  `Contest Create Month` COMMENT 'Records the month when the contest was created, useful for understanding the timeline of contest availability.',
  `User ID` COMMENT 'Represents the unique identifier for the user participating in the contest, essential for linking user actions to their respective entries.',
  `Team ID` COMMENT 'Indicates the unique identifier for the team that the user is part of in the contest, facilitating team-based tracking and performance analysis.',
  `User Create Date` COMMENT 'The date when the user was created, providing context for when the user first interacted with the system.',
  `Origin State` COMMENT 'Indicates the geographical state from which the user originated, which can be useful for demographic analysis and regional insights.',
  `Gender` COMMENT 'Represents the gender of the user, which can be important for understanding user demographics and tailoring services accordingly.',
  `Final Position Category` COMMENT 'Reflects the final ranking position category achieved by the user or team in the contest, providing insights into performance outcomes.',
  `Preferred Payment Method` COMMENT 'Identifies the payment method that the user prefers to use, which can help in optimizing payment options offered.',
  `Entry Amount Tier` COMMENT 'Represents the amount tier required to enter the contest, which is essential for participants to know before joining.',
  `Contest Count` COMMENT 'Distinct count of contests.',
  `Total Prize Pool` COMMENT 'Shows the total amount of money available to be won in the contest, giving participants an idea of the potential rewards.',
  `Total Entry Amount` COMMENT 'Represents the total amount required to enter the contest, which is essential for participants to know before joining.',
  `Average Entry Amount` COMMENT 'Represents the average amount required to enter the contest, which is essential for participants to know before joining.',
  `Total Fee Paid` COMMENT 'Denotes the total amount of fee that has been paid by the user to enter the contest, which is important for financial tracking and revenue analysis.',
  `Average Fee Paid` COMMENT 'Denotes the average amount of fee that has been paid by the user to enter the contest, which is important for financial tracking and revenue analysis.',
  `Unique Users Count` COMMENT 'Count of total distinct users.',
  `Unique Teams Count` COMMENT 'Count of total distinct teams.',
  `Total Wallet Amount` COMMENT "Represents the total amount of money available in the user's wallet, which can be relevant for understanding user financial capacity.",
  `Average Wallet Amount` COMMENT "Represents the average amount of money available in the user's wallet, which can be relevant for understanding user financial capacity.",
  `Total Wallet Transactions` COMMENT 'Indicates the total number of transactions made by the user in their wallet, useful for analyzing user engagement and activity levels.',
  `Average Transaction Amount` COMMENT 'Indicates the average amount of transactions made by the user in their wallet, useful for analyzing user engagement and activity levels.',
  `Total Net Amount` COMMENT "Represents the net amount after all transactions, which is important for assessing the user's financial standing.",
  `Total Discount Points` COMMENT 'Indicates the total number of discount points accumulated by the user, which can be relevant for loyalty programs and promotions.',
  `Average Payment Method Diversity` COMMENT 'Reflects the variety of payment methods used by the user, providing insights into user preferences and behavior.',
  `Win Rate` COMMENT 'The Win Rate measures the percentage of contest entries in which a user finished in the first position.',
  `Top 3 Rate` COMMENT 'The Top 3 Rate measures the percentage of contest entries in which a user finished in the top three positions (1, 2, or 3).'
  )
WITH METRICS
LANGUAGE YAML
COMMENT 'The table contains data related to contests, including details about the contests themselves and the users participating in them. Key information includes the sport type, contest type, entry fees, prize pools, and user demographics. This data can be used for analyzing contest performance, user engagement, and financial metrics related to contest entries and outcomes'
AS $$
version: 0.1

source: |
  SELECT 
      -- Contest Information
      c.contest_id,
      c.contest_sport,
      c.contest_type,
      c.contest_create_ts,
      c.entry_amount,
      c.prize_pool,
      
      -- Contest Participation Information
      ct.user_id,
      ct.team_id,
      ct.fee_paid,
      ct.final_position,
      
      -- User Demographics
      u.created_timestamp as user_created_timestamp,
      u.origin_state,
      u.gender,
      
      -- Aggregated Wallet Information (per user)
      wt.user_id as wallet_user_id,
      wt.total_wallet_amount,
      wt.total_wallet_transactions,
      wt.avg_transaction_amount,
      wt.total_net_amount,
      wt.total_discount_points,
      wt.preferred_payment_method,
      wt.payment_method_diversity

  FROM contests c
  LEFT JOIN contests_tx ct ON c.contest_id = ct.contest_id
  LEFT JOIN users u ON ct.user_id = u.user_id
  LEFT JOIN (
      -- Pre-aggregate wallet transactions per user
      SELECT 
          user_id,
          SUM(amount) as total_wallet_amount,
          COUNT(*) as total_wallet_transactions,
          AVG(amount) as avg_transaction_amount,
          SUM(net_amount) as total_net_amount,
          SUM(discount_points) as total_discount_points,
          MODE() WITHIN GROUP (ORDER BY instrument_type) as preferred_payment_method,
          COUNT(DISTINCT instrument_type) as payment_method_diversity
      FROM wallet_tx
      GROUP BY user_id
  ) wt ON u.user_id = wt.user_id

dimensions:
  - name: Contest ID
    expr: contest_id
  
  - name: Contest Sport
    expr: contest_sport
  
  - name: Contest Type
    expr: contest_type
  
  - name: Contest Create Date
    expr: DATE(contest_create_ts)
  
  - name: Contest Create Month
    expr: DATE_TRUNC('MONTH', contest_create_ts)
  
  - name: User ID
    expr: user_id
  
  - name: Team ID
    expr: team_id
  
  - name: User Create Date
    expr: DATE(user_created_timestamp)
  
  - name: Origin State
    expr: origin_state
  
  - name: Gender
    expr: gender
  
  - name: Final Position Category
    expr: CASE 
           WHEN final_position = 1 THEN 'Winner'
           WHEN final_position <= 3 THEN 'Top 3'
           WHEN final_position <= 10 THEN 'Top 10'
           ELSE 'Other'
         END
  
  - name: Preferred Payment Method
    expr: preferred_payment_method
  
  - name: Entry Amount Tier
    expr: CASE 
           WHEN entry_amount < 10 THEN 'Low ($0-$9)'
           WHEN entry_amount < 50 THEN 'Medium ($10-$49)'
           WHEN entry_amount < 100 THEN 'High ($50-$99)'
           ELSE 'Premium ($100+)'
         END

measures:
  - name: Contest Count
    expr: COUNT(DISTINCT contest_id)
  
  - name: Total Prize Pool
    expr: SUM(prize_pool)
  
  - name: Total Entry Amount
    expr: SUM(entry_amount)
  
  - name: Average Entry Amount
    expr: AVG(entry_amount)
  
  - name: Total Fee Paid
    expr: SUM(fee_paid)
  
  - name: Average Fee Paid
    expr: AVG(fee_paid)
  
  - name: Unique Users Count
    expr: COUNT(DISTINCT user_id)
  
  - name: Unique Teams Count
    expr: COUNT(DISTINCT team_id)
  
  - name: Total Wallet Amount
    expr: SUM(total_wallet_amount)
  
  - name: Average Wallet Amount
    expr: AVG(total_wallet_amount)
  
  - name: Total Wallet Transactions
    expr: SUM(total_wallet_transactions)
  
  - name: Average Transaction Amount
    expr: AVG(avg_transaction_amount)
  
  - name: Total Net Amount
    expr: SUM(total_net_amount)
  
  - name: Total Discount Points
    expr: SUM(total_discount_points)
  
  - name: Average Payment Method Diversity
    expr: AVG(payment_method_diversity)
  
  - name: Win Rate
    expr: COUNT(CASE WHEN final_position = 1 THEN 1 END) * 100.0 / COUNT(*)
  
  - name: Top 3 Rate
    expr: COUNT(CASE WHEN final_position <= 3 THEN 1 END) * 100.0 / COUNT(*)
$$;